In [1]:
from hyper import *
from alignment import *
from datasets.preprecossing import *
from core import *
from datasets.loading import *
from datasets.hc_dataset import *
from datasets.balance_dataset import *
from utils.linkage import *
from model.balancehc import balancehc

from utils.poincare import *
import scib
import shutil



In [2]:
import os
import numpy as np
import pandas as pd
import scib

import scanpy as sc
from pathlib import Path
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from os.path import exists
def get_count_data(adata,counts_location=None):
    
    data = adata.layers[counts_location].copy() if counts_location else adata.X.copy()
    if not isinstance(data, np.ndarray):
        data= data.toarray()
    data_df = pd.DataFrame(data,index=adata.obs_names,columns=adata.var_names).transpose()
    return data_df


def check_paths(output_folder,output_prefix=None):
    # Create relative path
    output_path = os.path.join(os.getcwd(), output_folder)

    # Make sure that the folder exists
    Path(output_path).mkdir(parents=True, exist_ok=True)

    if os.path.exists(os.path.join(output_path, f"{output_prefix}assigned_locations.csv")):
        print("\033[91mWARNING\033[0m: Running this will overwrite previous results, choose a new"
              " 'output_folder' or 'output_prefix'")

    return output_path
def remove_batch_effect(pseudo_bulk, bulk_adata, out_dir, project='',batch_effect=True):
    """
    Remove batch effect between pseudo_bulk and input bulk data.

    Parameters
    ----------
    pseudo_bulk : anndata.AnnData
        An :class:`~anndata.AnnData` containing the pseudo expression.
    bulk_adata : anndata.AnnData
        An :class:`~anndata.AnnData` containing the input expression.
    out_dir : string, optional
        The path to save the output file.
    project : string, optional
        The prefix of output file.
        
    Returns
    -------
    Returns the expression after removing batch effect.

    """
    out_dir = check_paths(out_dir+'/batch_effect')
    if batch_effect:
        if exists(f'{out_dir}/{project}_batch_effected.txt'):
            print(f'{out_dir}/{project}_batch_effected.txt already exists, skipping batch effect.')
            bulk_data = pd.read_csv(f"{out_dir}/{project}_batch_effected.txt",sep='\t').T
        else:
            
            save=True
            # check path, file will be stored in out_dir+'/batch_effect'
            pseudo_bulk_df = get_count_data(pseudo_bulk)
            input_bulk_df= get_count_data(bulk_adata)

            bulk = pd.concat([pseudo_bulk_df,input_bulk_df], axis=1)

            cells = np.append(pseudo_bulk.obs_names, bulk_adata.obs_names)
            batch = np.append(np.ones((1,len(pseudo_bulk.obs_names))), np.ones((1,len(bulk_adata.obs_names)))+1)
            if save:
                bulk.to_csv(out_dir+f"/{project}_before_batch_effected.txt",sep='\t')
            meta = pd.DataFrame({"batch": batch,"cells":cells})
            # get r script path
            robjects.r.source('./combat.R')
            pandas2ri.activate()
            robjects.r.run_combat(bulk, meta, out_dir, project)
            # stop auto trans from pandas to r dataframe
            pandas2ri.deactivate()
            # add layer
            if exists(f'{out_dir}/{project}_batch_effected.txt'):
                bulk_data = pd.read_csv(f"{out_dir}/{project}_batch_effected.txt",sep='\t').T
                print(bulk_data.shape)
            else:
                raise ValueError('The batch_effected data is not available')
        bulk_data.clip(lower=0,inplace=True)
        # print(pseudo_bulk)
        # print(pseudo_bulk.obs_names)
        pseudo_bulk.layers["batch_effected"] = bulk_data.loc[pseudo_bulk.obs_names,:].values
        bulk_adata.layers["batch_effected"] = bulk_data.loc[bulk_adata.obs_names,:].values
    else:
        pseudo_bulk_df = get_count_data(pseudo_bulk)
        input_bulk_df= get_count_data(bulk_adata)
        bulk = pd.concat([pseudo_bulk_df,input_bulk_df], axis=1)
        bulk.to_csv(out_dir+f"/{project}_batch_effected.txt",sep='\t')

    return pseudo_bulk,bulk_adata

python(46055) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(46056) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [3]:
cell_path1 = './datas/23/2/BRCA_GSE110686.h5ad'
folder_path1 = './datas/23/2/'
radius1 = 0
c1 =0
epoches1 =20
cell_path2 = "./datas/23/3/CRC_GSE146771.h5ad" 
folder_path2 = "./datas/23/3/" 
radius2 = 0
c2 =0
epoches2 = 20
contin = True
resolution=1
method='average'
alignment=1
n_pca=50
meta_col = 'Celltype..major.lineage.'
ms = False


In [4]:
# python run_sc.py -cp1 './datas/120/3/sample1_small.h5' -f1 "./datas/120/3/" -r1 25 -c1 0.1 -e1 10 -cp2 './datas/120/4/sample1_small.h5' -f2 "./datas/120/4/" -r2 25 -c2 0.1 -e2 10 --contin False --alignment 1 --resolution 1 --n_pca 500


In [5]:
if (contin==False) or ((os.path.exists(folder_path1+'merge_cell_data.csv') and os.path.exists(folder_path1 + 'merge_cell_meta.csv')) == False):
    loss1 = merge_by_radius(cell_path1,folder_path1,radius1,method,meta_col)
    print("cell meta score for dataset1: {}\n".format(loss1))
else:
    print("dataset1 find files and skip merging")



if(contin==False) or ((os.path.exists(folder_path2+'merge_cell_data.csv') and os.path.exists(folder_path2 + 'merge_cell_meta.csv')) == False):
    loss2 = merge_by_radius(cell_path2,folder_path2,radius2,method,meta_col)
    print("cell meta score for dataset2: {}".format(loss2))
else:
    print("dataset2 find files and skip merging")

    

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/orig.ident' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nCount_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nFeature_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_1' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_2' was written without encoding metadata.
  return 

cell meta score for dataset1: 1.0



/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/orig.ident' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nCount_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nFeature_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_1' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_2' was written without encoding metadata.
  return 

cell meta score for dataset2: 1.0


In [6]:
adata1 = sc.read_h5ad(folder_path1+ 'adata.h5ad')
adata2 = sc.read_h5ad(folder_path2+ 'adata.h5ad')


preprocessing_cluster(adata1,N_pcs=n_pca,resolution=resolution)
preprocessing_cluster(adata2,N_pcs=n_pca,resolution=resolution)

inter_gene = sort_data(adata1,adata2)

tmp1 = calculate_cluster_centroid_for_genes(adata1,inter_gene,folder_path1) 
tmp2 = calculate_cluster_centroid_for_genes(adata2,inter_gene,folder_path2)


meta_list1 = calculate_cluster_celltype(adata1);
meta_list2 = calculate_cluster_celltype(adata2);
calculate_meta_ori(folder_path1,adata1);
calculate_meta_ori(folder_path2,adata2);

2024-03-19 15:35:06.464953: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
if(contin==False) or ((os.path.exists(folder_path1 + 'dataxy.npy') and os.path.exists(folder_path1+'data1link.npy') and os.path.exists(folder_path1+'dataname.npy')) == False):
    get_Hyper_tree(folder_path1+'datas.data',1,tmp1.shape[1]+1,0,epoches1,10,save_path=folder_path1,mst=ms)
else:
    print("dataset1 find files and skip embedding");

if(contin==False) or ((os.path.exists(folder_path2 + 'dataxy.npy') and os.path.exists(folder_path2+'data1link.npy') and os.path.exists(folder_path1+'dataname.npy')) == False):
    get_Hyper_tree(folder_path2+'datas.data',1,tmp2.shape[1]+1,0,epoches2,10,save_path=folder_path2,mst=ms)
else:
    print("dataset2 find files and skip embedding")


./datas/23/2/datas.data length:10
Generating all pairs superset
	 Epoch 0 | average train loss: 0.966356
Optimization finished.
	 Epoch 1 | average train loss: 0.941207
Optimization finished.
	 Epoch 2 | average train loss: 0.941279
Optimization finished.
	 Epoch 3 | average train loss: 0.940414
Optimization finished.
	 Epoch 4 | average train loss: 0.940350
Optimization finished.
	 Epoch 5 | average train loss: 0.940468
Optimization finished.
	 Epoch 6 | average train loss: 0.941214
Optimization finished.
	 Epoch 7 | average train loss: 0.940824
Optimization finished.
	 Epoch 8 | average train loss: 0.940168
Optimization finished.
	 Epoch 9 | average train loss: 0.939446
Optimization finished.
	 Epoch 10 | average train loss: 0.939698
Optimization finished.
	 Epoch 11 | average train loss: 0.938994
Optimization finished.
	 Epoch 12 | average train loss: 0.940198
Optimization finished.
	 Epoch 13 | average train loss: 0.940589
Optimization finished.
	 Epoch 14 | average train loss: 0.9

/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/utils/mst.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ij = np.empty((n - 1, 2), dtype=np.int)
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/utils/mst.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current us

	 Epoch 0 | average train loss: 0.429346


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  origin_distance = hyp_dist(torch.tensor(self.original[i]),torch.tensor(self.original[j]));
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  origin_distance = hyp_dist(torch.tensor(self.original[int(nodef.son[0])]),torch.tensor(self.original[k])) + hyp_dist(torch.tensor(self.original[int(nodef.son[1])]),torch.tensor(self.original[k]));
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:92: UserWarning: To copy construct from a tensor, it is recommended 

	 Epoch 1 | average train loss: 0.423453
	 Epoch 2 | average train loss: 0.422780
	 Epoch 3 | average train loss: 0.422216
	 Epoch 4 | average train loss: 0.421836
	 Epoch 5 | average train loss: 0.421476
	 Epoch 6 | average train loss: 0.420833
	 Epoch 7 | average train loss: 0.419985
	 Epoch 8 | average train loss: 0.419198
	 Epoch 9 | average train loss: 0.419298
./datas/23/3/datas.data length:19
Generating all pairs superset
	 Epoch 0 | average train loss: 0.975638
Optimization finished.
	 Epoch 1 | average train loss: 0.969955
Optimization finished.
	 Epoch 2 | average train loss: 0.970281
Optimization finished.
	 Epoch 3 | average train loss: 0.969464
Optimization finished.
	 Epoch 4 | average train loss: 0.970345
Optimization finished.
	 Epoch 5 | average train loss: 0.969469
Optimization finished.
	 Epoch 6 | average train loss: 0.969194
Optimization finished.
	 Epoch 7 | average train loss: 0.968782
Optimization finished.
	 Epoch 8 | average train loss: 0.968777
Optimization f

In [15]:
nodes1 = build_hyper_tree_from_folder(folder_path1,ms)
nodes2 = build_hyper_tree_from_folder(folder_path2,ms)

# nodes1[0] = search_tree(nodes1[0],0.5,)
# nodes2[0] = search_tree(nodes2[0],15)
search_tree
def add_meta(now,meta_list,merge):
    if(int(now.name)<len(meta_list)):
        now.name= now.name +'_'+ meta_list[int(now.name)];
    merge.append(now)
    for i in now.son:
        add_meta(i,meta_list,merge)
nodes_merge1 = [];
nodes_merge2 = [];
add_meta(nodes1[0],meta_list1,nodes_merge1)
add_meta(nodes2[0],meta_list2,nodes_merge2)

        
rate = 0;        
if(alignment==1):
    rate,anslist,ans = run_alignment(nodes_merge1,nodes_merge2,folder_path1,folder_path2,meta_list1,meta_list2);
elif(alignment==2):
    rate = run_alignment_linear(nodes_merge1,nodes_merge2);

/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/alignment.py:749: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  father_name = father_name.astype(np.int)
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/alignment.py:749: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

The mininum cost for alignment is 35.747803121248495
The alignment edges list is [('18', '36'), ('17', '34'), ('16', '33'), ('14', '32'), ('12', '28'), ('8_CD4Tconv', '11_NK'), ('3_CD8Tex', '13_Malignant'), ('1_CD4Tconv', '25'), ('10', '22'), ('6_CD8T', '5_Treg'), ('0_CD4Tconv', '14_Mast'), ('15', '20'), ('5_CD4Tconv', '7_Plasma'), ('7_CD4Tconv', '1_NK'), ('13', '35'), ('9_Mono/Macro', '3_Malignant'), ('4_Treg', '30')]
average cost for one node:0.6383536271651516

correct alignment rate:0.0


In [21]:
(len(nodes1)+1)/2

10.0

In [20]:
(len(nodes2)+1)/2

19.0

In [24]:
nodes1

[18,
 13,
 4_Treg,
 9_Mono/Macro,
 17,
 15,
 7_CD4Tconv,
 11,
 2_CD8T,
 5_CD4Tconv,
 16,
 10,
 0_CD4Tconv,
 6_CD8T,
 14,
 1_CD4Tconv,
 12,
 3_CD8Tex,
 8_CD4Tconv]

In [27]:
def find_index(nodes,index):
    for i in nodes:
        if(int(i.name.split('_')[0]) ==index):
            return i;
ans = 0;
for i1,j1 in anslist:
    i =find_index(nodes1,i1);
    j = find_index(nodes2,j1)
    df = pd.DataFrame(
        {"A": i.value, "B":j.value})
    mincost = 1 - df.corr(method="spearman").iloc[0, 1]
    ans+=mincost

In [28]:
10+19 - 2*len(anslist) + ans

20.574741896758702

In [ ]:
    df = pd.DataFrame(
        {"A": i.value, "B":j.value})
    mincost = 1 - df.corr(method="spearman").iloc[0, 1]
    return mincost

In [16]:
anslist

[(8, 11), (3, 13), (6, 5), (0, 14), (5, 7), (7, 1), (9, 3)]

In [ ]:
anslist_dist = dict(ans)
anslist_dist.keys()
def search_lineage(now,path,anss):
    path.append(now.name)
    if(now.son==[]):
        anss.append(path);
        return
    
    for i in now.son:
        search_lineage(i,path.copy(),anss);
temp1 = []
search_lineage(nodes1[0],[],temp1)
temp1
route1 = []
route2 = []

for i in temp1:
    r1 = []
    r2 = []
    for j in i:
        if j in anslist_dist.keys():
            r1.append(j)
            r2.append(anslist_dist[j])
    route1.append(r1)
    route2.append(r2)
adata1.obs.index = [i+'_1' for i in adata1.obs.index]
adata2.obs.index = [i+'_2' for i in adata2.obs.index]

In [ ]:
score = [];


for i,j in zip(route1,route2):
    try:
        shutil.rmtree('./batch_effect/', ignore_errors=True)

        cells1 = [ ]
        cells2 = [ ]
        for k,t in zip(i,j):
            num1 = int(k.split('_')[0])
            num2 = int(t.split('_')[0])
            if(len(k.split('_'))>1):
                cells1.append(str(num1));
            if(len(t.split('_'))>1):
                cells2.append(str(num2));
        if(cells1==[] or cells2 ==[]):
            continue;
        
        sub_adata1 = adata1[adata1.obs['leiden'].isin(cells1),inter_gene].copy();
        sub_adata2 = adata2[adata2.obs['leiden'].isin(cells2),inter_gene].copy();

        adata1_after,adata2_after = remove_batch_effect(sub_adata1.copy(),sub_adata2.copy(),'./')
        
        sc.pp.neighbors(sub_adata1,use_rep='X')
        sc.tl.diffmap(sub_adata1)
        sub_adata1.uns['iroot'] = 0
        sc.tl.dpt(sub_adata1)
        
        
        sc.pp.neighbors(sub_adata2,use_rep='X')
        sc.tl.diffmap(sub_adata2)
        sub_adata2.uns['iroot'] = 0
        sc.tl.dpt(sub_adata2)
        
        
        adata1_after.obsm['batch_effected'] = adata1_after.layers['batch_effected']
        adata2_after.obsm['batch_effected'] = adata2_after.layers['batch_effected']
        
        sc.pp.neighbors(adata1_after,use_rep='batch_effected')
        sc.tl.diffmap(adata1_after)
        adata1_after.uns['iroot'] = 0
        sc.tl.dpt(adata1_after)

        sc.pp.neighbors(adata2_after,use_rep='batch_effected')
        sc.tl.diffmap(adata2_after)
        adata2_after.uns['iroot'] = 0
        sc.tl.dpt(adata2_after)
        
        score.append( scib.me.trajectory_conservation(sub_adata1, adata1_after, label_key="celltype"))
        score.append( scib.me.trajectory_conservation(sub_adata2, adata2_after, label_key="celltype"))
        shutil.rmtree('./batch_effect/', ignore_errors=True)
    except:
        print(cells1,cells2)
        pass;
score

In [ ]:
np.array(score).mean()

In [ ]:
0.761302636083028